# Activity 2.2 - Transfer Learning 

#### Objective(s):

This activity aims to introduce how to apply transfer learning 

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network 
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [10]:
from __future__ import print_function

import datetime
import tensorflow

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

Set the parameters

In [11]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [12]:

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps. 
* Use the model, training set, test set and number of classes as function parameters


In [13]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = tensorflow.keras.utils.to_categorical(train[1], num_classes)
    y_test = tensorflow.keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [14]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()



Create two datasets 
* one with digits below 5
* one with 5 and above

In [15]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [16]:

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [17]:

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [18]:

model = Sequential(feature_layers + classification_layers)

Check the model summary

In [19]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

 Train the  model on the digits 5,6,7,8,9

In [20]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 1s 36us/sample - loss: 1.6047 - acc: 0.2254 - val_loss: 1.5891 - val_acc: 0.2726
Epoch 2/5
29404/29404 [==============================] - 1s 30us/sample - loss: 1.5835 - acc: 0.2779 - val_loss: 1.5654 - val_acc: 0.4016
Epoch 3/5
29404/29404 [==============================] - 1s 30us/sample - loss: 1.5624 - acc: 0.3344 - val_loss: 1.5394 - val_acc: 0.5063
Epoch 4/5
29404/29404 [==============================] - 1s 32us/sample - loss: 1.5379 - acc: 0.3993 - val_loss: 1.5105 - val_acc: 0.5690
Epoch 5/5
29404/29404 [==============================] - 1s 31us/sample - loss: 1.5118 - acc: 0.4564 - val_loss: 1.4786 - val_acc: 0.6170
Training time: 0:00:05.171704
Test score: 1.4785524483105696
Test accuracy: 0.6169512


Freeze only the feature layers

In [21]:

for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

Train again the model using the 0 to 4 digits

In [23]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 1s 25us/sample - loss: 1.5741 - acc: 0.3180 - val_loss: 1.5480 - val_acc: 0.3898
Epoch 2/5
30596/30596 [==============================] - 1s 23us/sample - loss: 1.5420 - acc: 0.3789 - val_loss: 1.5118 - val_acc: 0.4682
Epoch 3/5
30596/30596 [==============================] - 1s 25us/sample - loss: 1.5073 - acc: 0.4554 - val_loss: 1.4762 - val_acc: 0.6174
Epoch 4/5
30596/30596 [==============================] - 1s 21us/sample - loss: 1.4756 - acc: 0.5201 - val_loss: 1.4410 - val_acc: 0.7373
Epoch 5/5
30596/30596 [==============================] - 1s 22us/sample - loss: 1.4421 - acc: 0.5842 - val_loss: 1.4058 - val_acc: 0.8046
Training time: 0:00:03.756417
Test score: 1.4057998115113097
Test accuracy: 0.8046313


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [24]:
### Reverse model that trains on 0-4 then finetune on 5-9
model_reverse = Sequential(feature_layers + classification_layers)
model_reverse.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [25]:
## Train on numbers 0-4
train_model(model_reverse,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 1s 28us/sample - loss: 1.4105 - acc: 0.6384 - val_loss: 1.3723 - val_acc: 0.8418
Epoch 2/5
30596/30596 [==============================] - 1s 22us/sample - loss: 1.3785 - acc: 0.6869 - val_loss: 1.3387 - val_acc: 0.8599
Epoch 3/5
30596/30596 [==============================] - 1s 26us/sample - loss: 1.3454 - acc: 0.7231 - val_loss: 1.3050 - val_acc: 0.8723
Epoch 4/5
30596/30596 [==============================] - 1s 24us/sample - loss: 1.3149 - acc: 0.7481 - val_loss: 1.2711 - val_acc: 0.8827
Epoch 5/5
30596/30596 [==============================] - 1s 22us/sample - loss: 1.2829 - acc: 0.7759 - val_loss: 1.2375 - val_acc: 0.8899
Training time: 0:00:04.022838
Test score: 1.2374604431704617
Test accuracy: 0.8898619


In [26]:
for l in feature_layers:
    l.trainable = False
    
model_reverse.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_4 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)             

In [27]:
train_model(model_reverse,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 1s 32us/sample - loss: 1.4602 - acc: 0.4263 - val_loss: 1.4274 - val_acc: 0.4913
Epoch 2/5
29404/29404 [==============================] - 1s 22us/sample - loss: 1.4309 - acc: 0.4694 - val_loss: 1.3962 - val_acc: 0.5600
Epoch 3/5
29404/29404 [==============================] - 1s 21us/sample - loss: 1.4017 - acc: 0.5144 - val_loss: 1.3661 - val_acc: 0.6124
Epoch 4/5
29404/29404 [==============================] - 1s 21us/sample - loss: 1.3755 - acc: 0.5548 - val_loss: 1.3372 - val_acc: 0.6579
Epoch 5/5
29404/29404 [==============================] - 1s 22us/sample - loss: 1.3488 - acc: 0.5851 - val_loss: 1.3095 - val_acc: 0.6861
Training time: 0:00:03.651321
Test score: 1.309515796480687
Test accuracy: 0.6860728


#### Conclusion

#type your answer here